[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/google-ai-edge/model-explorer/blob/main/example_colabs/custom_data_overlay_demo.ipynb)

# Install packages

In [1]:
# Install tf-nightly & model-explorer.
!pip install tf-nightly
!pip install --no-deps ai-edge-model-explorer ai-edge-model-explorer-adapter

# Install kagglehub (will be used in the next step to download a model)
!pip install kagglehub --no-deps

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 556.3/556.3 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 53.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.0/322.0 kB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 79.5 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.5
    Uninstalling protobuf-5.29.5:
      Successfully uninstalled protobuf-5.29.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-ai-generativelanguage 0.6.15 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<6.0.0dev,>=3.20.2, but you have protobuf 6.32.1 which is incompatible.
tensorflow 2.19.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<6.0.0dev,>=3.20.3, but you have protobuf 6.32.1 which is incompatible.
grpcio-

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 44.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.2/90.2 MB 8.1 MB/s eta 0:00:00


# Download MobileNet v3 from Kaggle

In [2]:
import kagglehub

# This demo uses MobileNet v3, but you can use other models as well
path = kagglehub.model_download(
    "google/mobilenet-v3/tfLite/large-075-224-classification"
)
model_path = f"{path}/1.tflite"


  0%|          | 0.00/15.3M [00:00<?, ?B/s]
 33%|███▎      | 5.00M/15.3M [00:00<00:00, 50.4MB/s]
100%|██████████| 15.3M/15.3M [00:00<00:00, 52.1MB/s]


# Run the model with test data

In [3]:
import numpy as np
import tensorflow as tf

# Load the TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path=model_path)
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Generate random input data.
for input_detail in input_details:
  input_shape = input_detail['shape']
  input_data = np.array(
      np.random.random_sample(input_shape), dtype=input_detail['dtype']
  )
  interpreter.set_tensor(input_detail['index'], input_data)

# Run the model on random input data.
interpreter.invoke()

# Examine the output data (optional)
for output_detail in output_details:
  print(f"Output for {output_detail['name']}")
  output_data = interpreter.get_tensor(output_detail['index'])
  print(output_data)

NotFoundError: /usr/local/lib/python3.12/dist-packages/tensorflow/core/kernels/libtfkernel_sobol_op.so: undefined symbol: _ZN10tensorflow15TensorShapeBaseINS_11TensorShapeEEC2EN4absl12lts_202308024SpanIKlEE

# Prepare per-op benchmarking data for Model Explorer
## Step 1: Run the benchmark

In [ ]:
!mkdir -p /tmp/data

%cd /tmp/data

CPU_PROFILING_PROTO_PATH = "/tmp/data/mv3-cpu-op-profile.pb"

# In this example, we're using profiling data from Android's Benchmarking tools
# that has already been mapped (outside of this Colab) to the Model Explorer schema.

# You can overlay per-op data of your choice by following the instructions at
# https://github.com/google/model-explorer/wiki/2.-User-Guide#custom-node-data

%env MODEL_PATH=$model_path
%env CPU_PROFILING_PROTO_PATH=$CPU_PROFILING_PROTO_PATH

# Download the tflite model benchmark binary.
!wget -nc https://storage.googleapis.com/tensorflow-nightly-public/prod/tensorflow/release/lite/tools/nightly/latest/linux_x86-64_benchmark_model
!chmod +x /tmp/data/linux_x86-64_benchmark_model

# Run the benchmark locally only using CPU kernels with op_profiling enabled.
!./linux_x86-64_benchmark_model --graph=$MODEL_PATH --use_xnnpack=false --num_threads=4 --enable_op_profiling=true --op_profiling_output_mode=proto --op_profiling_output_file=$CPU_PROFILING_PROTO_PATH

## Step 2: Generate the per-op profiling JSON using benchmark results.

In [4]:
import collections
from collections.abc import Sequence
import json
import os
import re
from typing import Any, List

from tensorflow.lite.profiling.proto import profiling_info_pb2

_PER_OP_LATENCY_JSON_TYPE = "per_op_latency"
_OP_TYPE_JSON_TYPE = "op_type"

_MODEL_EXPLORER_JSON_TYPES = [_PER_OP_LATENCY_JSON_TYPE, _OP_TYPE_JSON_TYPE]


def get_op_profile_json(
    op_profile: profiling_info_pb2.OpProfileData,
    model_explorer_json_type: str,
) -> dict[str, Any]:
  """Generates the Model Explorer json for the op profile.

  Args:
    op_profile: profiling_info_pb2.OpProfileData
    model_explorer_json_type: Type of model explorer json to generate.

  Returns:
    Model explorer json for the op profile.

  Raises:
    ValueError: If the op profile name is not in the expected format.
    ValueError: If the model explorer json type is not supported.
  """
  op_profile_key_re = re.findall(r":(\d+)$", op_profile.name)
  if not op_profile_key_re:
    raise ValueError("Op profile name is not in the expected format.")
  op_profile_key = op_profile_key_re[0]

  if model_explorer_json_type == _PER_OP_LATENCY_JSON_TYPE:
    return {
        op_profile_key: {
            "value": op_profile.inference_microseconds.avg / 1000.0
        }
    }
  elif model_explorer_json_type == _OP_TYPE_JSON_TYPE:
    return {op_profile_key: {"value": op_profile.node_type}}
  else:
    raise ValueError(
        "Unsupported model explorer json type: %s" % model_explorer_json_type
    )


def generate_model_explorer_json(
    benchmark_profiling_proto_paths: List[str],
    output_path: str,
    model_explorer_json_type: str,
) -> dict[str, Any]:
  """Generates the Model Explorer json for the benchmark profiling proto."""
  if not benchmark_profiling_proto_paths:
    raise ValueError("At least one profiling proto path should be provided.")

  if model_explorer_json_type not in _MODEL_EXPLORER_JSON_TYPES:
    raise ValueError(
        f"Unsupported model explorer json type: {model_explorer_json_type}"
    )

  output_json = collections.defaultdict(dict)
  for proto_path in benchmark_profiling_proto_paths:
    if not os.path.isfile(proto_path):
      raise ValueError(f"File {proto_path} does not exist.")

    with open(proto_path, "rb") as f:
      benchmark_profiling_proto = (
          profiling_info_pb2.BenchmarkProfilingData.FromString(f.read())
      )
      for (
          subgraph_profile
      ) in benchmark_profiling_proto.runtime_profile.subgraph_profiles:
        subgraph_profile_json = {}
        for op_profile in subgraph_profile.per_op_profiles:
          subgraph_profile_json.update(
              get_op_profile_json(op_profile, model_explorer_json_type)
          )
        output_json[subgraph_profile.subgraph_name][
            "results"
        ] = subgraph_profile_json

        if model_explorer_json_type == _PER_OP_LATENCY_JSON_TYPE:
          output_json[subgraph_profile.subgraph_name]["gradient"] = [
              {"stop": 0, "bgColor": "green"},
              {"stop": 0.33, "bgColor": "yellow"},
              {"stop": 0.67, "bgColor": "orange"},
              {"stop": 1, "bgColor": "red"},
          ]

  if output_path:
    with open(output_path, "w") as f:
      json.dump(output_json, f)
  else:
    print(json.dumps(output_json, indent=2))


CPU_PROFILING_JSON_PATH = "/tmp/data/mv3-cpu-op-profile.json"

# Generate pure CPU per-op profiling JSON.
generate_model_explorer_json(
    [CPU_PROFILING_PROTO_PATH],
    CPU_PROFILING_JSON_PATH,
    _PER_OP_LATENCY_JSON_TYPE,
)

# Download the XNNPACK per-op profiling JSON from storage.
!wget -nc https://storage.googleapis.com/tfweb/model-explorer-demo/mv3-xnnpack-op-profile.json
XNNPACK_PROFILING_JSON_PATH = "/tmp/data/mv3-xnnpack-op-profile.json"

NameError: name 'CPU_PROFILING_PROTO_PATH' is not defined

# Visualize the model with per op latency

In [ ]:
import model_explorer

config = model_explorer.config()
(
    config.add_model_from_path(model_path)
    .add_node_data_from_path(CPU_PROFILING_JSON_PATH)
    .add_node_data_from_path(XNNPACK_PROFILING_JSON_PATH)
)

model_explorer.visualize_from_config(config)